# 1. O modelo de dados do Python

## 1.2. Um baralho pythônico


### 1.2.1. Classe `Card`


`Card` é um `namedtuple` que representa uma carta de baralho.

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

O "sete belo" é conhecido como "beer card" nos EUA:

In [2]:
beer_card = Card('7', 'diamonds')
beer_card

Card(rank='7', suit='diamonds')

In [3]:
beer_card.rank, beer_card.suit

('7', 'diamonds')

In [4]:
beer_card[0], beer_card[1]

('7', 'diamonds')

Uma limitação de `namedtuple` é não permitir a declaração de métodos como em uma classe.

Não é recomendado, mas é possível acrescentar métodos por [monkey-patch](https://en.wikipedia.org/wiki/Monkey_patch): você define uma função e atribui ela à classe, da mesma forma que criamos atributos em instâncias:

In [5]:
def card_to_str(card):
    return '%s of %s' % card

card_to_str(beer_card)

'7 of diamonds'

Agora podemos atribuir a função à classe com o nome especial `__str__`:

In [6]:
Card.__str__ = card_to_str
print(beer_card)

7 of diamonds


### 1.2.2. Classe `FrenchDeck`

`FrenchDeck` é uma classe que representa um "baralho francês", o tipo mais comum no Brasil, com 52 cartas em 4 naipes de 13 cartas.

In [7]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

Veja as operações possíveis com um baralho:

In [8]:
deck = FrenchDeck()
len(deck)

52

In [9]:
deck[3]

Card(rank='5', suit='spades')

In [10]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

**Nota**: nesse contexto o Python não usa o `__str__` de `Card`. Se você exibir com `print` uma carta de cada vez, daí o `__str__` é usado.

Como `FrenchDeck` implementa `__len__` e `__getitem__`, Python entende que é uma `Sequence` (sequência), então o operador `in` funciona:

In [11]:
Card('Q', 'hearts') in deck

True

#### Exercício

Crie uma carta que não existe, e verifique que o `in` devolve `False`

In [12]:
Card('H', 'hearts') in deck

False

O laço `for` sabe lidar com sequências:

In [13]:
for card in deck:
    print(card)

2 of spades
3 of spades
4 of spades
5 of spades
6 of spades
7 of spades
8 of spades
9 of spades
10 of spades
J of spades
Q of spades
K of spades
A of spades
2 of diamonds
3 of diamonds
4 of diamonds
5 of diamonds
6 of diamonds
7 of diamonds
8 of diamonds
9 of diamonds
10 of diamonds
J of diamonds
Q of diamonds
K of diamonds
A of diamonds
2 of clubs
3 of clubs
4 of clubs
5 of clubs
6 of clubs
7 of clubs
8 of clubs
9 of clubs
10 of clubs
J of clubs
Q of clubs
K of clubs
A of clubs
2 of hearts
3 of hearts
4 of hearts
5 of hearts
6 of hearts
7 of hearts
8 of hearts
9 of hearts
10 of hearts
J of hearts
Q of hearts
K of hearts
A of hearts


Muitos métodos da biblioteca padrão lidam com sequências:

In [14]:
from random import choice
choice(deck)

Card(rank='J', suit='hearts')

Mas a função `shuffle` não funciona. Resolver isso será um exercício logo mais:

In [15]:
from random import shuffle

# This should raise a TypeError
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

Podemos usar `sorted` para percorrer o baralho em pela ordem de comparação dos elementos das tuplas `Card`:

In [16]:
for card in sorted(deck):
    print(card)

10 of clubs
10 of diamonds
10 of hearts
10 of spades
2 of clubs
2 of diamonds
2 of hearts
2 of spades
3 of clubs
3 of diamonds
3 of hearts
3 of spades
4 of clubs
4 of diamonds
4 of hearts
4 of spades
5 of clubs
5 of diamonds
5 of hearts
5 of spades
6 of clubs
6 of diamonds
6 of hearts
6 of spades
7 of clubs
7 of diamonds
7 of hearts
7 of spades
8 of clubs
8 of diamonds
8 of hearts
8 of spades
9 of clubs
9 of diamonds
9 of hearts
9 of spades
A of clubs
A of diamonds
A of hearts
A of spades
J of clubs
J of diamonds
J of hearts
J of spades
K of clubs
K of diamonds
K of hearts
K of spades
Q of clubs
Q of diamonds
Q of hearts
Q of spades


Podemos criar uma função que estabelece um critério de ordenação melhor:

In [17]:
suit_values = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high_ordering(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [18]:
spades_high_ordering(Card('2', 'clubs'))

0

In [19]:
spades_high_ordering(Card('A', 'spades'))

51

Daí usamos a função como parâmetro `key` em `sorted`. Várias outras funções da biblioteca padrão que envolvem comparação de itens aceitam o parâmetro `key`.

In [20]:
for card in sorted(deck, key=spades_high_ordering):
    print(card)

2 of clubs
2 of diamonds
2 of hearts
2 of spades
3 of clubs
3 of diamonds
3 of hearts
3 of spades
4 of clubs
4 of diamonds
4 of hearts
4 of spades
5 of clubs
5 of diamonds
5 of hearts
5 of spades
6 of clubs
6 of diamonds
6 of hearts
6 of spades
7 of clubs
7 of diamonds
7 of hearts
7 of spades
8 of clubs
8 of diamonds
8 of hearts
8 of spades
9 of clubs
9 of diamonds
9 of hearts
9 of spades
10 of clubs
10 of diamonds
10 of hearts
10 of spades
J of clubs
J of diamonds
J of hearts
J of spades
Q of clubs
Q of diamonds
Q of hearts
Q of spades
K of clubs
K of diamonds
K of hearts
K of spades
A of clubs
A of diamonds
A of hearts
A of spades


#### Exercício

Defina uma nova ordem que classifique as cartas primeiro por naipe e depois por valor, de forma que todos os paus venham primeiro, seguidos por todos os ouros, etc.

In [26]:
suit_values = {'clubs': 0, 'diamonds': 1, 'hearts': 2, 'spades': 3}
def suits_first_ordering(card):
    rank_index = FrenchDeck.ranks.index(card.rank)
    return suit_values[card.suit] * len(FrenchDeck.ranks) + rank_index

for card in sorted(deck, key=suits_first_ordering):
    print(card)

2 of clubs
3 of clubs
4 of clubs
5 of clubs
6 of clubs
7 of clubs
8 of clubs
9 of clubs
10 of clubs
J of clubs
Q of clubs
K of clubs
A of clubs
2 of diamonds
3 of diamonds
4 of diamonds
5 of diamonds
6 of diamonds
7 of diamonds
8 of diamonds
9 of diamonds
10 of diamonds
J of diamonds
Q of diamonds
K of diamonds
A of diamonds
2 of hearts
3 of hearts
4 of hearts
5 of hearts
6 of hearts
7 of hearts
8 of hearts
9 of hearts
10 of hearts
J of hearts
Q of hearts
K of hearts
A of hearts
2 of spades
3 of spades
4 of spades
5 of spades
6 of spades
7 of spades
8 of spades
9 of spades
10 of spades
J of spades
Q of spades
K of spades
A of spades


#### Exercício

Escreva um método chamado `setcard` que pega um baralho, um índice e uma carta e atribui a carta ao baralho na posição dada.

Em seguida, faça um monkey-patch em `FrenchDeck` para fornecer `__setitem__` como método. Teste atribuindo uma nova carta assim:

```meu_baralho[0] = Card('A', 'spades')```

Então tenter embaralhar usando `random.shuffle`.

In [47]:
def setcard(deck, position, card):
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = setcard
meu_baralho = FrenchDeck()
meu_baralho[0] = Card('A', 'spades')

import random
random.shuffle(meu_baralho)

print(meu_baralho[:5])

#### Exercício bônus

A operação de fatiamento `x[a:b]` normalmente devolve uma instância da mesma classe de `x`.

Será necessário alterar `FrenchDeck` para fazer isso acontecer? Como?

**Dica 1:** O compilador de Python transforma `x[a:b]` em `x.__getitem__(slice(a, b))`.  

**Dica 2:** Será preciso mexer no construtor de `FrenchDeck`, aceitando uma lista opcional de cartas.

In [51]:
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self, cards=None):
        self._cards = cards or [Card(rank, suit) 
                                for suit in self.suits
                                for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        match position:
            case slice():
                return type(self)(self._cards[position])
            case _:
                return self._cards[position]

deck = FrenchDeck()
new_deck = deck[:5]

print(new_deck)
for card in new_deck:
    print(card)

2 of spades
3 of spades
4 of spades
5 of spades
6 of spades
